# Milvus Database

ilvus is a powerful vector database tailored for processing and searching extensive vector data.</br>
For more information please see: https://milvus.io/

In [3]:
# Install Python SDK
# %pip install pymilvus

  Using cached pymilvus-2.4.3-py3-none-any.whl.metadata (5.3 kB)
  Using cached grpcio-1.63.0-cp310-cp310-macosx_12_0_universal2.whl.metadata (3.2 kB)
  Using cached protobuf-5.27.0-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached environs-9.5.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached ujson-5.10.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (9.3 kB)
  Using cached pandas-2.2.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached milvus_lite-2.4.6-py3-none-macosx_11_0_arm64.whl.metadata (5.6 kB)
  Using cached marshmallow-3.21.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pymilvus-2.4.3-py3-none-any.whl (194 kB)
Using cached environs-9.5.0-py2.py3-none-any.whl (12 kB)
Using cached grpcio-1.63.0-cp310-cp310-macosx_12_0_universal2.whl (10.1 MB)
Using cached milvus_lite-2.4.6-py3-none-macosx_1

In [4]:
import pymilvus
pymilvus.__version__

'2.4.3'

Since we are using lite version. Create local database or connect to local database (as a file).

In [5]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

In [6]:
client.create_collection(
    collection_name="demo_collection",
    dimension=768  # The vectors we will use in this demo has 768 dimensions
)

# Represent text with vectors
First, install the model library. This package includes essential ML tools such as PyTorch. The package download may take some time if your local environment has never installed PyTorch.

In [7]:
# %pip install "pymilvus[model]"


  Using cached milvus_model-0.2.3-py3-none-any.whl.metadata (1.6 kB)
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached onnxruntime-1.18.0-cp310-cp310-macosx_11_0_universal2.whl.metadata (4.2 kB)
  Using cached scipy-1.13.1-cp310-cp310-macosx_12_0_arm64.whl.metadata (60 kB)
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached huggingface_hub-0.23.2-py3-none-any.whl.metadata (12 kB)
  Using cached regex-2024.5.15-cp310-cp310-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached safetensors-0.4.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached sympy-1.12.1-py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-1

In [8]:
from pymilvus import model

# If connection to https://huggingface.co/ failed, uncomment the following path
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# This will download a small embedding model "paraphrase-albert-small-v2" (~50MB).
embedding_fn = model.DefaultEmbeddingFunction()

# Text strings to search from.
docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]

vectors = embedding_fn.encode_documents(docs)
# The output vector has 768 dimensions, matching the collection that we just created.
print("Dim:", embedding_fn.dim, vectors[0].shape)  # Dim: 768 (768,)

# Each entity has id, vector representation, raw text, and a subject label that we use
# to demo metadata filtering later.
data = [ {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"} for i in range(len(vectors)) ]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


/opt/homebrew/anaconda3/envs/data-engineering/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
/opt/homebrew/anaconda3/envs/data-engineering/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Dim: 768 (768,)
Data has 3 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 768


## [Alternative] Use Fake Data

In [ ]:
# import random

# # Text strings to search from.
# docs = [
#     "Artificial intelligence was founded as an academic discipline in 1956.",
#     "Alan Turing was the first person to conduct substantial research in AI.",
#     "Born in Maida Vale, London, Turing was raised in southern England.",
# ]
# # Use fake representation with random vectors (768 dimension).
# vectors = [ [ random.uniform(-1, 1) for _ in range(768) ] for _ in docs ]
# data = [ {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"} for i in range(len(vectors)) ]

# print("Data has", len(data), "entities, each with fields: ", data[0].keys())
# print("Vector dim:", len(data[0]["vector"]))


# Insert Data

In [9]:
res = client.insert(
    collection_name="demo_collection",
    data=data
)

print(res)

{'insert_count': 3, 'ids': [0, 1, 2], 'cost': 0}


# Semantic Search
Now we can do semantic searches by representing the search query text as vector, and conduct vector similarity search on Milvus.

## Vector search
Milvus accepts one or multiple vector search requests at the same time. The value of the query_vectors variable is a list of vectors, where each vector is an array of float numbers.

In [10]:
query_vectors = embedding_fn.encode_queries([ "Who is Alan Turing?" ])

# Alternative when using fake data
# If you don't have the embedding function you can use a fake vector to finish the demo:
# query_vectors = [ [ random.uniform(-1, 1) for _ in range(768) ] ]

res = client.search(
    collection_name="demo_collection", # target collection
    data=query_vectors,                # query vectors
    limit=2,                           # number of returned entities
    output_fields=["text", "subject"], # specifies fields to be returned
)

print(res)


data: ["[{'id': 2, 'distance': 0.5859944820404053, 'entity': {'text': 'Born in Maida Vale, London, Turing was raised in southern England.', 'subject': 'history'}}]"] , extra_info: {'cost': 0}


# Vector Search with Metadata Filtering

You can also conduct vector search while considering the values of the metadata (called "scalar" fields in Milvus, as scalar refers to non-vector data). This is done with a filter expression specifying certain criteria. Let's see how to search and filter with the subject field in the following example.

In [11]:
# Insert more docs in another subject.
docs = [
    "Machine learning has been used for drug design.",
    "Computational synthesis with AI algorithms predicts molecular properties.",
    "DDR1 is involved in cancers and fibrosis.",
]
vectors = embedding_fn.encode_documents(docs)
data = [ {"id": 2+i, "vector": vectors[i], "text": docs[i], "subject": "biology"} for i in range(len(vectors))]

client.insert(
    collection_name="demo_collection",
    data=data
)

# This will exclude any text in "history" subject despite close to the query vector.
res = client.search(
    collection_name="demo_collection",
    data=embedding_fn.encode_queries([ "tell me AI related information" ]),
    filter="subject == 'biology'",
    limit=2,
    output_fields=["text", "subject"],
)

print(res)


data: ["[{'id': 3, 'distance': 0.2703056335449219, 'entity': {'text': 'Computational synthesis with AI algorithms predicts molecular properties.', 'subject': 'biology'}}]"] , extra_info: {'cost': 0}


## Query
A `query()` is an operation that retrieves all entities matching a cretria, such as a filter expression or matching some ids.

For example, the following cell retrieving all entities whose scalar field has a particular value.

In [12]:
res = client.query(
    collection_name="demo_collection",
    filter="subject == 'history'",
    output_fields=["text", "subject"],
)

Directly retrieve entities by primary key.

In [13]:
res = client.query(
    collection_name="demo_collection",
    ids=[0,2],
    output_fields=["vector", "text", "subject"]
)


# Delete Entity
To purge data: delete entities specifying the primary key or delete all entities matching a particular filter expression.

In [14]:
# Delete entity by PK
res = client.delete(collection_name="demo_collection", ids=[0, 2])

print(res)

# Delete entities by a filter expression
res = client.delete(
    collection_name="demo_collection",
    filter="subject == 'biology'",
)

print(res)

[0, 0, 2, 2, 2]
[3, 3, 4, 4]


# Loading Existing Data
Since all data of Milvus Lite is stored in a local file, you can load all data into memory even after the program terminates, by creating a `MilvusClient` with the existing file. For example, this will recover the collections from "milvus_demo.db" file and continue to write data into it.

In [15]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

# Drop Collections


Delete all the data in a collection

In [16]:
client.drop_collection(collection_name="demo_collection")

Connect to external Milvus instance, for example, docker local instance

In [17]:
client = MilvusClient(uri="http://localhost:19530", token="root:Milvus")

Failed to create new connection using: e02dd78671a046409b7c25e35306f446


MilvusException: <MilvusException: (code=2, message=Fail connecting to server on localhost:19530, illegal connection params or server unavailable)>